In [1]:
import pandas as pd
import numpy as np
import catboost

Read input data

In [2]:
df = pd.read_csv('train.csv')
df_tst = pd.read_csv('test.csv')

Fill unknown values on mean value

In [3]:
for d in df,df_tst:
    d['Cabin'].fillna('Unknown',inplace=True)
    d['Embarked'].fillna('Unknown',inplace=True)

Drop too personal column

In [4]:
cols = list(df.columns)
cols.remove('Survived')
cols.remove('PassengerId')

CatBoost can work with categorial features, we don't need to bring "Sex" and "Embarked" to integer values.

In [5]:
df_male = df[df.Sex == 'male']
df_female = df[df.Sex == 'female']

df_tst_male = df_tst[df_tst.Sex == 'male']
df_tst_female = df_tst[df_tst.Sex == 'female']

In [6]:
cbc_male = catboost.CatBoostClassifier(random_seed=0).fit(df_male[cols].values,df_male['Survived'],cat_features=[0,1,2,6,8,9])
cbc_female = catboost.CatBoostClassifier(random_seed=0).fit(df_female[cols].values,df_female['Survived'],cat_features=[0,1,2,6,8,9])

In [7]:
p_male = cbc_male.predict(df_tst_male[cols].values).astype(int)
p_female = cbc_female.predict(df_tst_female[cols].values).astype(int)

In [8]:
submission_male = pd.DataFrame({
        "PassengerId": df_tst_male["PassengerId"],
        "Survived": p_male
    })
submission_female = pd.DataFrame({
        "PassengerId": df_tst_female["PassengerId"],
        "Survived": p_female
    })
submission = submission_male.append(submission_female, ignore_index=True)
submission.to_csv("submission.csv", index=False)